### Checking and cleaning the data from summer microbiology surveys

#### There are two consecutive years of data. Stored in different excel tables and in diferent formats. The data needs to be aggregated and transformed in the following ways:

1. Ensure date formats are consistent in both sets
2. Identify and separate incubation times
3. Identify unique sample days per year
4. Ensure the precipitation data is consistent in format with the micro-biology data
5. Develop categories and grouping methods to analyse and present the results
5. Store and separate the results in a proper directory structure


#### The data should be usable either as .csv or JSON format for a variety of tasks.

1. The results maybe transformed further along the chain (grouping/aggregating/summarising)
2. This should be considered the "starting point"

In [1]:
import pandas as pd
import numpy as np
import json
import re

### Make a fucntion to write JSON files

#### We will be pushing most of our work to JSON or csv

1. The built in pandas function will not always work
2. Most of what is pushed is already in dictionary format

In [2]:
def make_json(a, b):
    with open(a, 'wb') as f:
        f.write(json.dumps(b).encode('utf-8'))

### Import the data and clean it up

1. Format the dates
2. Fix column names (ensure they all have the same syntax)
3. Identify any records that need to be excluded for other reasons (nan values, weather, etc..)

In [3]:
# get the data
a = pd.read_csv('data/2017_Data.csv')

In [4]:
# this is a duplicate value in the DF, the data was moved and then added as a record)
# so we need to get rid of that:
a.drop(12, inplace=True)
# there is a column name that doesn't match the others
a.rename(columns={'p3_fluo_halo_colonies':'P3_fluo_halo_colonies'}, inplace=True)

#### Format dates 2017 data:

In [5]:
# format the 2017 dates
# for whatever reason the pd.datetime did not recognize the date 17.06.12
# it flipped it to : 17.12.06 ---- which won't work
# so use some list index and str properties to make this right

b = a['Date'].copy()
new_dates = []
new_dic = {}
for x in b:
    year = '20'+str(x[6:])
    day = x[:2]
    month = x[3:5]
    new_d = year + '/' + str(month) + '/' + str(day)
    new_dates.append({x:new_d})
    new_dic.update({x:new_d})
new_dates[:10]

a['Date'] = a['Date'].map(new_dic)

In [6]:
# The dates were put into a "string" format
# This is good for reading but not for doing calculations
# Use to_datetime to convert to a timestamp
a['Date'] = pd.to_datetime(a['Date'],format='%Y/%m/%d' )


In [7]:
a[(a.Location == 'MRD') & (a.Date == '2017-07-24')]
# this record needs to go or we need to reset the sample number
# check the sampling notes to see why
# so even though the sampling was ineffective there is still a value in the samples column

,Date,Location,medium,Samples,Sampling_Notes,Water_temp,Plating_notes,Temp_incubation,P1_qty_sample,Image_24h_fluo_plate_one,...,P3_48h_big_blue,P3_48h_med_blue,P3_48h_green,P3_48h_turq,P3_48h_pink,P3_48h_other,Comments_p3_48h,latitude,longitude,Location_Image
45,2017-07-24,MRD,easy_gel,3,"Waves at sight, sampling not effective",0,No comments,37,4.0,none,...,0.0,0.0,0.0,0.0,0.0,0.0,No comments,46.431662,6.908962,1500895656638.jpg


In [8]:
# go ahead and change that now
a.loc[(a.Location == 'MRD') & (a.Date == '2017-07-24'), "Samples"] = 0

In [9]:
a.loc[(a.Location == 'MRD') & (a.Date == '2017-07-24'), "Sampling_Notes"] = "Waves at site, sampling not effective"

In [10]:
# Query that record to make sure it is ok

In [11]:
a[(a.Location == 'MRD') & (a.Date == '2017-07-24')]

,Date,Location,medium,Samples,Sampling_Notes,Water_temp,Plating_notes,Temp_incubation,P1_qty_sample,Image_24h_fluo_plate_one,...,P3_48h_big_blue,P3_48h_med_blue,P3_48h_green,P3_48h_turq,P3_48h_pink,P3_48h_other,Comments_p3_48h,latitude,longitude,Location_Image
45,2017-07-24,MRD,easy_gel,0,"Waves at site, sampling not effective",0,No comments,37,4.0,none,...,0.0,0.0,0.0,0.0,0.0,0.0,No comments,46.431662,6.908962,1500895656638.jpg


In [12]:
# The data will be served in JSON
# convert here and read back to make sure everything is ok
a.to_json('data/JSON/2017ColonyCounts.json', orient='index')
a.to_csv('data/CSV/2017ColonyCounts.csv')

In [13]:
# reading the data back in should produce no errors:
a = pd.read_json('data/JSON/2017ColonyCounts.json', orient='index')

### This is "record data" each row corresponds to an event:
#### Each event is a water sample from a location on a particular day

1. For each event there are three samples
2. Those three samples each get their own selective medium plate
3. Each plate has a subtotal "colony count"
4. The average of the subtotals is the "Total colony count" for that day and location
5. The bacterial species are identified by the color of the colony
6. The colonies are counted at 48h and 24h

### Initially we need to be able to develop a method to "call" individual values:

1. Per location
2. Per date
3. Per species
4. Per incubation time

#### Then we can chart and compare results across locations and from one year to another.


### There are two incubation times for each sample - 24 and 48 hours:

1. Seperate the records
2. Export the data as a csv

#### Date groupings, by week 2017

In [14]:
# group the dates by week number
# first get a sorted list by date
b = list(a['Date'].unique())

# then convert the dataes to a timestamp that can be used by python/numpy
b = pd.to_datetime(b, format='%Y/%m/%d')

# week number is how we will measure the "distance" from the "Jazz Event"
week_number = ['Week one', 'Week two', 'Week three', 'Week four', 'Week five', 'Week six', 'Week seven', 'Week eight']

# pair the week number with the corresponding date combine in a dictionary:
weeks = dict(zip(week_number, b))

# the out put looks like this:
# {'Week one': Timestamp('2017-06-12 00:00:00'),...'Week eight': Timestamp('2017-07-31 00:00:00')}

# inverse the key:value pairs:
weeks_2 = dict(zip(b, week_number))
# !! Attention:
# the time stamp works here but it is a hassel in js
# and we can allways use == to group by date strings

In [15]:
# The date in string format will be used in JSON output
# Easy to parse in JS with Date.parse method:

def convert_to_string(a):
    d = {}
    for key, value in a.items():
        b = value.strftime('%Y-%m-%d')
        c = {key:b}
        d.update(c)
    return d
    
weeks_2017 = convert_to_string(weeks)
dates_2017 = {value:key for key, value in weeks_2017.items()}

# the output looks like this:
#{'2017-06-12': 'Week one',...'2017-07-31': 'Week eight'}

# write both versions to JSON and save them as utitlities

In [16]:
make_json('data/utilities/dateWeek2017JsonObj.json',dates_2017)
make_json('data/utilities/weekDate2017JsonObj.json',weeks_2017)

### Get and check 2016 data:

In [17]:
a16 = pd.read_csv('data/2016_Data.csv')
a16.drop('Unnamed: 0', axis=1, inplace=True)
a16.to_json('data/JSON/2016ColonyCounts.json', orient='index')

#### Date groupings, by week 2016

This is the same process as the dates for 2017, for an explanation see blocks above.

In [18]:
b_2016 = a16['Date'].unique()
b_2016.sort()
b_2016 = pd.to_datetime(b_2016,format='%Y-%m-%d')

In [19]:
weeks_2016 = dict(zip(week_number, b_2016))
dates_2016 = dict(zip(b_2016, week_number))
week_date16 = convert_to_string(weeks_2016)
date_week16 = {value:key for key, value in week_date16.items()}
make_json('data/utilities/weekDate2016JsonObj.json',week_date16)
make_json('data/utilities/dateWeek2016JsonObj.json',date_week16 )

### Mediums used to culture bacteria

1. Create a list of the different culture mediums that were used

In [20]:
# a list of mediums is important
mediums = list(a['medium'].unique().copy())
# push that to JSON
make_json('data/utilities/mediums2017.json', mediums)

### Locations of interest

1. Create a list of the sample locations to be compared

In [21]:
places_48 = ['MRD', 'VNX', 'SVT']
make_json('data/utilities/locations.json',places_48)

### Link column names to colony color

1. indifferent of platenumber
2. used to label charts 
3. Easier to remember

In [22]:
# create plate variables
p_one_48 = ['P1_fluo_halo_colonies','P1_48h_big_blue','P1_48h_med_blue','P1_48h_green',
            'P1_48h_turq','P1_48h_pink','P1_48h_other']
p_two_48 = ['P2_fluo_halo_colonies','P2_48h_big_blue','P2_48h_med_blue',
            'P2_48h_green','P2_48h_turq','P2_48h_pink','P2_48h_other']
p_three_48 = ['P3_fluo_halo_colonies','P3_48h_big_blue','P3_48h_med_blue',
              'P3_48h_green','P3_48h_turq','P3_48h_pink','P3_48h_other']
# create key for charting
import re
q = re.compile('fluo_halo', re.IGNORECASE)
def make_keys(a):
    f={}
    for h in a:
        for b, c in enumerate(h):
            if q.search(h[b]):
                d = 'UV Fluo'
                e = {h[0]:d}
                f.update(e)
            else:
                d = h[b][7:]
                e = {h[b]:d}
                f.update(e)
    return f
colony_map = make_keys([p_one_48, p_two_48, p_three_48])
make_json('data/utilities/colony_map.json', colony_map)

In [23]:
colony_map


{'P1_fluo_halo_colonies': 'UV Fluo',
 'P1_48h_big_blue': 'big_blue',
 'P1_48h_med_blue': 'med_blue',
 'P1_48h_green': 'green',
 'P1_48h_turq': 'turq',
 'P1_48h_pink': 'pink',
 'P1_48h_other': 'other',
 'P2_fluo_halo_colonies': 'UV Fluo',
 'P2_48h_big_blue': 'big_blue',
 'P2_48h_med_blue': 'med_blue',
 'P2_48h_green': 'green',
 'P2_48h_turq': 'turq',
 'P2_48h_pink': 'pink',
 'P2_48h_other': 'other',
 'P3_fluo_halo_colonies': 'UV Fluo',
 'P3_48h_big_blue': 'big_blue',
 'P3_48h_med_blue': 'med_blue',
 'P3_48h_green': 'green',
 'P3_48h_turq': 'turq',
 'P3_48h_pink': 'pink',
 'P3_48h_other': 'other'}

#### This create user friendly names mapped to orignal dataframe column names:

In [24]:
# the 'map' can be used to convert
# column names to something more concise
# can use index location or counts to call labels
# so that means that
colony_map[p_two_48[1]]

'big_blue'

#### Test that the same index number calls the same color group across all plates

In [25]:
# to check that the same color group is called for each plate
# grab a random number and use it to
# index the lists created from the columns
# these will be used as labels later to call data out
# the output  should be the same for all values x[i] where i = np.random.choice(len(list of names))
plates_four_eight =  [p_one_48, p_two_48, p_three_48]
plates_2four_eight = [p_one_48, p_two_48]
plates_1four_eight = [p_one_48]

In [26]:
plates_1four_eight

[['P1_fluo_halo_colonies',
  'P1_48h_big_blue',
  'P1_48h_med_blue',
  'P1_48h_green',
  'P1_48h_turq',
  'P1_48h_pink',
  'P1_48h_other']]

### Calling color groups

1. We need to be able to call color groups by plate number and/or incubation time
2. this needs to be passed on in JSON format also

#### Ensure that with one index call all the results for a particular species can be collected

In [27]:
# use the above property to make color groups
# each color represents a type/family of bacteria
# we need two calculate the results per color group
# in the preceding step they are grouped by plate
a_number = np.arange(len(plates_four_eight[0]))
def make_groups(x, q):
    b=[]
    for s in x:
        a = []
        for y in q:
            a.append(y[s])
        b.append(a)
    return b
three_plate_48 = make_groups(a_number, plates_four_eight)
two_plate_48 = make_groups(np.arange(len(plates_2four_eight[0])), plates_2four_eight)
one_plate_48 = make_groups(np.arange(len(plates_1four_eight[0])), plates_1four_eight)

# take a look
# they should all be the same group:
three_plate_48[2]#< ---------- this is good
# perfect now I can call a color group by the number of samples

['P1_48h_med_blue', 'P2_48h_med_blue', 'P3_48h_med_blue']

### 24 hour groups
### Re-use the functions from the 48 hour group

In [28]:
p_one_24 = ['P1_fluo_halo_colonies', 'P1_24h_big_blue','P1_24h_med_blue',
            'P1_24h_green', 'P1_24h_turq', 'P1_24h_pink', 'P1_24h_other']
p_two_24 = ['P2_fluo_halo_colonies','P2_24h_big_blue','P2_24h_med_blue',
            'P2_24h_green', 'P2_24h_turq', 'P2_24h_pink','P2_24h_other',]
p_three_24 = ['P3_fluo_halo_colonies', 'P3_24h_big_blue', 'P3_24h_med_blue',
              'P3_24h_green', 'P3_24h_turq', 'P3_24h_pink','P3_24h_other']

colony_map_24 = make_keys([p_one_24, p_two_24, p_three_24] )


In [29]:
plates_two_four = [p_one_24, p_two_24, p_three_24]
plates_2four_four = [p_one_24, p_two_24]
plates_1four_four = [p_one_24]
# check_indexes(plates_two_four)

In [30]:
three_plate_24 = make_groups(np.arange(len(plates_two_four[0])), plates_two_four)
two_plate_24 = make_groups(np.arange(len(plates_2four_four[0])), plates_2four_four)
one_plate_24 = make_groups(np.arange(len(plates_1four_four[0])), plates_1four_four)

# take a look
# they should all be the same group:
two_plate_24[2]
# perfect now I can call a color group by the number of samples

['P1_24h_med_blue', 'P2_24h_med_blue']

In [31]:
three_plate_24[2]

['P1_24h_med_blue', 'P2_24h_med_blue', 'P3_24h_med_blue']

In [32]:
make_json('data/utilities/threeP2417.json', three_plate_24)
make_json('data/utilities/twoP2417.json', two_plate_24)
make_json('data/utilities/oneP2417.json', one_plate_24)

### 2016 groups

In [33]:
p_one_2016 = ['P1_24h_big_blue','P1_24h_med_blue','P1_24h_turq', 'P1_24h_pink', 'P1_24h_other']
p_two_2016 = ['P2_24h_big_blue','P2_24h_med_blue','P2_24h_turq', 'P2_24h_pink','P2_24h_other',]
p_three_2016 = ['P3_24h_big_blue', 'P3_24h_med_blue','P3_24h_turq', 'P3_24h_pink','P3_24h_other']

In [34]:
# make some keys for the 2016 data
# there is no fluo
def make_keys_16(a):
    f={}
    for h in a:
        for b, c in enumerate(h):
            d = h[b][7:]
            e = {h[b]:d}
            f.update(e)
    return f

In [35]:
colony_map_16 = make_keys_16([p_one_2016, p_two_2016, p_three_2016])

In [36]:
colony_map_16

{'P1_24h_big_blue': 'big_blue',
 'P1_24h_med_blue': 'med_blue',
 'P1_24h_turq': 'turq',
 'P1_24h_pink': 'pink',
 'P1_24h_other': 'other',
 'P2_24h_big_blue': 'big_blue',
 'P2_24h_med_blue': 'med_blue',
 'P2_24h_turq': 'turq',
 'P2_24h_pink': 'pink',
 'P2_24h_other': 'other',
 'P3_24h_big_blue': 'big_blue',
 'P3_24h_med_blue': 'med_blue',
 'P3_24h_turq': 'turq',
 'P3_24h_pink': 'pink',
 'P3_24h_other': 'other'}

In [37]:
plates_16 = [p_one_2016, p_two_2016, p_three_2016]
three_plate_16 = make_groups(np.arange(len(plates_16[0])), plates_16)

In [38]:
colony_map_16[p_two_2016[1]]

'med_blue'

In [39]:
three_plate_16[4] 

['P1_24h_other', 'P2_24h_other', 'P3_24h_other']

In [40]:
make_json('data/utilities/threeP2416.json', three_plate_16)

In [41]:
# okay good to go

## Rain data 2017

In [42]:
# we need to add rain values for the sampling period
# let's see what we got from the CSO
aa = pd.read_csv('data/rainfall.csv', header=None)
aa.iloc[:2]
# actually this came as a table embedded in a word doc
# try incorporating that into you analysis
# either way this won't work

,0,1,2,3,4
0,06/01/18,76°/58°,0 IN,0 IN,
1,06/02/18,78°/56°,0 IN,0 IN,


In [43]:
# the columns are in string format
# can't do math with that (the dates maybe)
# drop what we don't need
# this may not be the fastest way, but it is a direct route
bb = aa[[0, 2]].copy()

In [44]:
bb[:10]

,0,2
0,06/01/18,0 IN
1,06/02/18,0 IN
2,06/03/18,0.42 IN
3,06/04/18,0.08 IN
4,06/05/18,0 IN
5,06/06/18,0.57 IN
6,06/07/18,0 IN
7,06/08/18,0 IN
8,06/09/18,0 IN
9,06/10/18,0 IN


In [45]:
# type(bb[2][0]) ---> string
# so will split and catch only the number
cc = bb[2].str.split(expand=True)
# only need the first column of this new df
cc.rename(columns={0:'Rain'}, inplace=True)

In [46]:
cc[:10]

,Rain,1
0,0,IN
1,0,IN
2,0.42,IN
3,0.08,IN
4,0,IN
5,0.57,IN
6,0,IN
7,0,IN
8,0,IN
9,0,IN


In [47]:
dd = pd.concat([bb, cc], axis=1)
dd.drop([2,1], axis=1, inplace=True)
dd['Rain'] = dd['Rain'].astype(float)
print(dd.iloc[:1], type(dd['Rain'][0]), type(dd[0][0]))
# okay the date column is all that is left to do
# notice the dates are given as 2018
# that is because the original file has the dates as day/month
# the spreadsheet software defaults to the current year

          0  Rain
0  06/01/18   0.0 <class 'numpy.float64'> <class 'str'>


In [48]:
# first rename the column
dd.rename(columns={0:'Date'}, inplace=True)
# check again before we go down some long road
dd['Date'][0]

'06/01/18'

In [49]:
# see how this converts using datetime method
pd.to_datetime(dd['Date'][0])
# if i am lucky it will give me the year

Timestamp('2018-06-01 00:00:00')

In [50]:
# The year is 2018, so we need to change that
dd['Date'] = pd.to_datetime(dd['Date'])
dd['real_date'] = dd['Date'] - pd.Timedelta(days=365)
dd['real_date'][0]

Timestamp('2017-06-01 00:00:00')

In [51]:
# now i can get the dates i need for the study
# first get rid of some columns
dd.drop('Date', axis=1, inplace=True)
dd.columns

Index(['Rain', 'real_date'], dtype='object')

In [52]:
# use the date list created at the begining
# to define the date range
dd_d = 'real_date'
# give this a shot
# dd[(dd[dd_d] >= b[0]) & (dd[dd_d] <= b[7])]
# that will work
# therefore
d_rain = dd[(dd[dd_d] >= '2017-06-05') & (dd[dd_d] <= b[7])]
# move the columns around
d_rain = d_rain[[dd_d, 'Rain']]
# take a look
d_rain.iloc[:4]
# we are in business

,real_date,Rain
4,2017-06-05,0.00
5,2017-06-06,0.57
6,2017-06-07,0.00
7,2017-06-08,0.00


In [53]:
r_2017 = d_rain.copy()

In [54]:
r_2017['Date']=r_2017['real_date'].dt.strftime("%Y-%m-%d")
r_2017.drop('real_date',axis=1, inplace=True)
r_2017.to_json('data/JSON/rain2017.json', orient='index')
r_2017.to_csv('data/CSV/rain2017.csv')

In [55]:
d_rain['real_date'][4]

Timestamp('2017-06-05 00:00:00')

### 2016 rain data

In [56]:
rain_2016 = pd.read_csv('data/rainfall2016.csv', header=None)
rain_2016.rename(columns={0:'Date', 1:'Rain'}, inplace=True)
rain_2016.to_json('data/JSON/rain2016.json', orient='index')

In [57]:
rain_2016.iloc[:5]

,Date,Rain
0,2016-6-21,0.00
1,2016-06-22,0.00
2,2016-06-23,0.00
3,2016-06-24,1.66
4,2016-06-25,7.36
